In [34]:
import base64
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State
from dash_table.Format import Format, Group, Scheme, Symbol

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps

#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from AnimalShelter import AnimalShelter





###########################
# Data Manipulation / Model
###########################
# FIX ME change for your username and password and CRUD Python module name
username = "Administrator"
password = "Nicholas10"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.read({}))
dfwr = pd.DataFrame.from_records(shelter.read({"animal_type":"Dog","breed":
             {"$in":["Labrador Retriever Mix","Chesapeake Bay Retriever","Newfoundland"]},
             "sex_upon_outcome":"Intact Female",
             "age_upon_outcome_in_weeks":{"$gte":26},
             "age_upon_outcome_in_weeks":{"$lte":156}}))
dfmwr = pd.DataFrame.from_records(shelter.read({"animal_type":"Dog","breed":
             {"$in":["German Shepherd","Alaskan Malamute","Old English Sheepdog","Siberian Huskey","Rottweiler"]},
             "sex_upon_outcome":"Intact Male",
             "age_upon_outcome_in_weeks":{"$gte":26},
             "age_upon_outcome_in_weeks":{"$lte":156}}))
dfdit = pd.DataFrame.from_records(shelter.read({"animal_type":"Dog","breed":
             {"$in":["Doberman Pinscher","German Shepherd","Golden Retriever","Bloodhound","Rottweiler"]},
             "sex_upon_outcome":"Intact Female",
             "age_upon_outcome_in_weeks":{"$gte":20},
             "age_upon_outcome_in_weeks":{"$lte":300}}))



#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Header("Animal Shelter AAC Database by Nicholas Salvato"),
    html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode())),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard'))),
    html.Hr(),
    html.Div(
        
#FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.
    dcc.RadioItems(
            id='filter-type',
            options=[
                {'label': 'Water Rescue', 'value': 'WR'},
                {'label': 'Mountain or Wilderness Rescue', 'value': 'MWR'},
                {'label': 'Disaster or Individual Tracking', 'value': 'DIT'},
                {'label': 'Reset', 'value':'R'}
            ],
            value='R',
            labelStyle={'display': 'inline-block'}
            ),


    ),
    html.Hr(),
    dt.DataTable(
        id='datatable-id',
        columns=[
            {'name': 'Animal ID', 'id': 'animal_id', 'type': 'text', 'editable': False},
            {'name': 'Type', 'id': 'animal_type', 'type': 'text'},
            {'name': 'Breed', 'id': 'breed', 'type': 'text'},
            {'name': 'Color', 'id': 'color', 'type': 'text'},
            {'name': 'Birthday', 'id': 'date_of_birth', 'type': 'date'},
            {'name': 'DateTime', 'id': 'datetime', 'type': 'datetime'},
            {'name': 'monthyear', 'id': 'monthyear', 'type': 'datetime'},
            {'name': 'Animal Name', 'id': 'name', 'type': 'text'},
            {'name': 'Outcome', 'id': 'outcome_type', 'type': 'text'},
            {'name': 'Outcome Subtype', 'id': 'outcome_subtype', 'type': 'text'},
            {'name': 'Sex', 'id': 'sex_upon_outcome', 'type': 'text'},
            {'name': 'Age', 'id': 'age_upon_outcome_in_weeks', 'type': 'numeric',
             'format': Format(
                scheme=Scheme.fixed, 
                precision=2,
                decimal_delimiter='.')},
            {'name': 'Latitude', 'id': 'location_lat', 'type':'numeric',
            'format': Format(
                scheme=Scheme.fixed, 
                precision=3,
                group=Group.yes,
                groups=3,
                group_delimiter=' ',
                decimal_delimiter='.')},
            {'name': 'Longitude', 'id': 'location_long', 'type': 'numeric',
            'format': Format(
                scheme=Scheme.fixed, 
                precision=3,
                group=Group.yes,
                groups=3,
                group_delimiter=' ',
                decimal_delimiter='.')}
            
            
        ],
        data=df.to_dict('records'),
#FIXME: Set up the features for your interactive data table to make it user-friendly for your client
#If you completed the Module Six Assignment, you can copy in the code you created here 
        #editable = True,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        row_selectable="single",
        selected_rows=[],
        page_action="native",
        page_current= 0,
        page_size= 10,
        style_as_list_view=True,
    style_cell={'padding': '5px'},
    style_header={
        'backgroundColor': 'white',
        'fontWeight': 'bold'
    },
    style_cell_conditional=[
        {
            'if': {'column_id': c},
            'textAlign': 'left'
        } for c in ['Date', 'Region']
    ]
    ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
# ### FIX ME Add code to filter interactive data table with MongoDB queries

        
        if filter_type == 'R':
            new_df = df
        elif filter_type == 'WR':
            new_df = dfwr
        elif filter_type == 'MWR':
            new_df = dfmwr
        elif filter_type == 'DIT':
            new_df = dfdit
        else:
            new_df = df
            
        data = new_df.to_dict('records')
        
        columns=[
            {'name': 'Animal ID', 'id': 'animal_id', 'type': 'text', 'editable': False},
            {'name': 'Type', 'id': 'animal_type', 'type': 'text'},
            {'name': 'Breed', 'id': 'breed', 'type': 'text'},
            {'name': 'Color', 'id': 'color', 'type': 'text'},
            {'name': 'Birthday', 'id': 'date_of_birth', 'type': 'date'},
            {'name': 'DateTime', 'id': 'datetime', 'type': 'datetime'},
            {'name': 'monthyear', 'id': 'monthyear', 'type': 'datetime'},
            {'name': 'Animal Name', 'id': 'name', 'type': 'text'},
            {'name': 'Outcome', 'id': 'outcome_type', 'type': 'text'},
            {'name': 'Outcome Subtype', 'id': 'outcome_subtype', 'type': 'text'},
            {'name': 'Sex', 'id': 'sex_upon_outcome', 'type': 'text'},
            {'name': 'Age', 'id': 'age_upon_outcome_in_weeks', 'type': 'numeric',
             'format': Format(
                scheme=Scheme.fixed, 
                precision=2,
                decimal_delimiter='.')},
            {'name': 'Latitude', 'id': 'location_lat', 'type':'numeric',
            'format': Format(
                scheme=Scheme.fixed, 
                precision=3,
                group=Group.yes,
                groups=3,
                group_delimiter=' ',
                decimal_delimiter='.')},
            {'name': 'Longitude', 'id': 'location_long', 'type': 'numeric',
            'format': Format(
                scheme=Scheme.fixed, 
                precision=3,
                group=Group.yes,
                groups=3,
                group_delimiter=' ',
                decimal_delimiter='.')}  
        ]
        
        return (data, columns)




@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    ###FIX ME ####
#     add code for chart of your choice (e.g. pie chart) #
        dff = pd.DataFrame.from_dict(viewData)
        # count by breed occurrence
        new_df = dff.breed.value_counts()
        new_df.name = "breed"
        new_df = new_df.to_frame()
        new_df.reset_index(inplace=True)
        
        # return updated graph data
        return [
        dcc.Graph(
        figure = px.pie(new_df, values="breed", names="index")
        )
        ]
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):
#FIXME: Add in the code for your geolocation chart
#If you completed the Module Six Assignment, you can copy in the code you created here.
    dff = pd.DataFrame.from_dict(viewData)
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            dl.Marker(position=[dff.loc[0,'location_lat'],dff.loc[0,'location_long']], children=[
                dl.Tooltip(dff.iloc[0,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[1,9])
                ])
            ])
        ])
    ]



app